In [0]:
# Cellule 1 : Configuration Azure Data Lake Gen2
STORAGE_ACCOUNT_NAME = "datalakecertifimpe"
STORAGE_ACCOUNT_KEY = "xxxxxxxxxxxxxxxxxx" # Remplace par ta clé
CONTAINER_NAME = "data-gouv"


print("✅ Configuration chargée")
print(f"📦 Compte de stockage : {STORAGE_ACCOUNT_NAME}")
print(f"📂 Conteneur : {CONTAINER_NAME}")

# Cellule 2 : Configuration Spark pour Azure Data Lake Gen2
spark.conf.set(
    f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
    STORAGE_ACCOUNT_KEY
)

print("✅ Configuration Spark pour ADLS Gen2 effectuée")

# Cellule 3 : Lister tous les fichiers disponibles dans le dossier sirene
print(f"\n{'='*60}")
print(f"📂 Recherche des fichiers Parquet dans le dossier sirene...")

try:
    # Lister le conteneur racine
    root_path = f"abfss://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/"
    print(f"\n🔍 Contenu du conteneur '{CONTAINER_NAME}' :")
    
    root_files = dbutils.fs.ls(root_path)
    for item in root_files:
        item_type = "📁" if item.isDir() else "📄"
        size_mb = item.size / (1024**2) if item.size > 0 else 0
        print(f"  {item_type} {item.name} ({size_mb:.2f} MB)")
    
    # Essayer le dossier sirene/raw si il existe
    try:
        sirene_path = f"abfss://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/sirene/raw/"
        print(f"\n🔍 Contenu du dossier 'sirene/raw/' :")
        sirene_files = dbutils.fs.ls(sirene_path)
        for item in sirene_files:
            item_type = "📁" if item.isDir() else "📄"
            size_mb = item.size / (1024**2) if item.size > 0 else 0
            print(f"  {item_type} {item.name} ({size_mb:.2f} MB)")
    except Exception as e:
        print(f"⚠️  Le dossier 'sirene/raw/' n'existe pas encore")
        
except Exception as e:
    print(f"❌ Erreur lors du listage : {str(e)}")

# Cellule 4 : Définir le chemin du fichier
# Le chemin trouvé dans la cellule 3
adls_path = f"abfss://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/sirene/raw/sirene_stock_etablissement_20251114_183648.parquet"

print(f"📍 Chemin du fichier : {adls_path}")
print(f"✅ Chemin configuré pour le fichier Parquet")

# Cellule 5 : Lecture du fichier Parquet
print(f"\n{'='*60}")
print(f"📖 Lecture des données depuis ADLS Gen2...")

try:
    df = spark.read.parquet(adls_path)
    
    print(f"✅ Lecture réussie !")
    print(f"📋 Nombre de colonnes : {len(df.columns)}")
    
    # Compter le nombre de lignes (peut prendre du temps sur de gros datasets)
    print(f"\n⏳ Comptage du nombre total de lignes en cours...")
    total_rows = df.count()
    print(f"📊 Nombre total de lignes : {total_rows:,}")
    
except Exception as e:
    print(f"❌ Erreur lors de la lecture : {str(e)}")
    print(f"\n💡 Retourne à la cellule 3 pour vérifier le chemin correct du fichier")
    df = None

# Cellule 6 : Afficher le schéma
if df is not None:
    print(f"\n{'='*60}")
    print(f"📋 Schéma du DataFrame :")
    df.printSchema()
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

# Cellule 7 : Afficher les 10 premières lignes
if df is not None:
    print(f"\n{'='*60}")
    print(f"🔎 Les 10 premières lignes :")
    df.show(10, truncate=False)
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

# Cellule 8 : Aperçu avec limite de caractères (plus lisible)
if df is not None:
    print(f"\n{'='*60}")
    print(f"🔎 Les 10 premières lignes (tronqué à 50 caractères) :")
    df.show(10, truncate=50)
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

# Cellule 9 : Afficher les noms des colonnes
if df is not None:
    print(f"\n{'='*60}")
    print(f"📋 Liste des colonnes ({len(df.columns)} colonnes) :")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

# Cellule 10 : Statistiques de base sur quelques colonnes numériques
if df is not None:
    print(f"\n{'='*60}")
    print(f"📈 Statistiques de base :")
    df.describe().show()
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

# Cellule 11 : Conversion en Pandas pour affichage (ATTENTION: seulement 10 lignes)
if df is not None:
    print(f"\n{'='*60}")
    print(f"🐼 Conversion en Pandas (10 premières lignes) :")
    df_pandas = df.limit(10).toPandas()
    print(df_pandas)
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

# Cellule 12 : Sélectionner des colonnes spécifiques
if df is not None:
    print(f"\n{'='*60}")
    print(f"🎯 Sélection de colonnes spécifiques (exemple) :")
    
    # Remplace par les noms de colonnes qui t'intéressent
    # Exemple : si tu as des colonnes comme 'siren', 'siret', 'denomination'
    # df_selected = df.select('siren', 'siret', 'denomination')
    # df_selected.show(10, truncate=50)
    
    print("💡 Décommente et adapte la sélection ci-dessus selon tes besoins")
else:
    print("⚠️  DataFrame non chargé. Vérifie les cellules précédentes.")

✅ Configuration chargée
📦 Compte de stockage : datalakecertifimpe
📂 Conteneur : data-gouv
✅ Configuration Spark pour ADLS Gen2 effectuée

📂 Recherche des fichiers Parquet dans le dossier sirene...

🔍 Contenu du conteneur 'data-gouv' :
  📁 data_NAF/ (0.00 MB)
  📁 data_to_BDD_data_gouv_city_france/ (0.00 MB)
  📁 scrap_auto/ (0.00 MB)
  📁 sirene/ (0.00 MB)

🔍 Contenu du dossier 'sirene/raw/' :
  📁 sirene_stock_etablissement_20251114_183648.parquet/ (0.00 MB)
📍 Chemin du fichier : abfss://data-gouv@datalakecertifimpe.dfs.core.windows.net/sirene/raw/sirene_stock_etablissement_20251114_183648.parquet
✅ Chemin configuré pour le fichier Parquet

📖 Lecture des données depuis ADLS Gen2...
✅ Lecture réussie !
📋 Nombre de colonnes : 53

⏳ Comptage du nombre total de lignes en cours...
📊 Nombre total de lignes : 42,151,993

📋 Schéma du DataFrame :
root
 |-- siren: string (nullable = true)
 |-- nic: long (nullable = true)
 |-- siret: string (nullable = true)
 |-- statutDiffusionEtablissement: string